## Preprocessing and Feature Extraction

In [41]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
from textblob import TextBlob 

from nltk.corpus import stopwords
from nltk.corpus import twitter_samples
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
import nltk

from argparse import Namespace
from functools import partial
from pathlib import Path
from pprint import pprint

import os
import pickle
import random
import re
import string

In [42]:
df = pd.read_csv("../Data/Ukraine Tweets.csv")

## Sentiment Analysis:

Precounting of features: Length, Hashtags, URLs and Mentions

In [44]:
df['tweet_length'] = df['rendered_content'].apply(len)

In [58]:
df['num_mentions'] = df['rendered_content'].apply(lambda x: x.count('@'))

In [56]:
df['num_hashtags'] = df['rendered_content'].apply(lambda x: x.count('#'))

In [60]:
df['num_urls'] = df['rendered_content'].apply(lambda x: x.count('https'))

Steps taken:
- Converting emojis to text
- We decide to remove all the mentions and hashtagged words, as these will be analysed separately
- Remove Links, as these don't contribute to SA
- Conducting the SA on our preprocessed data

Creating a new column so that we can see the adjusted tweet and original versiom

In [64]:
df.insert(loc=6,
          column='Adjusted Tweet',
          value=df['rendered_content'])

Converting emojis to text

In [7]:
import emot
import emoji

def demote(text):
    text = emoji.demojize(text)
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(demote)

ModuleNotFoundError: No module named 'emot'

Removing mentions and hashtagged words

In [65]:
import re

def remove_mentions_hashtags(text):
    text = re.sub("@[A-Za-z0-9_]+","", text)
    text = re.sub("#[A-Za-z0-9_]+","", text)
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(remove_mentions_hashtags)

Removing links

In [66]:
import re

def remove_links(text):
    text = re.sub('http://\S+|https://\S+', '', text)
    text = re.sub('http[s]?://\S+', '', text)
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r'bit.ly/\S+', '', text) # remove bitly links
    text = text.strip('[link]') # remove [links]
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(remove_links)

Insert Polarity Score Column

In [67]:
df.insert(loc=7,
          column='Polarity Score',
          value=df['Adjusted Tweet'])

Sentiment Analysis using NLTK's VADER

In [68]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

def sentiment_analysis(text):  
    text = sia.polarity_scores(text)
    return text

df['Polarity Score'] = df['Polarity Score'].apply(sentiment_analysis)
    

LookupError: 
**********************************************************************
  Resource [93mvader_lexicon[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('vader_lexicon')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93msentiment/vader_lexicon.zip/vader_lexicon/vader_lexicon.txt[0m

  Searched in:
    - 'C:\\Users\\ekmho/nltk_data'
    - 'C:\\Users\\ekmho\\anaconda3\\nltk_data'
    - 'C:\\Users\\ekmho\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\ekmho\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\ekmho\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


Creating columns for:
- Negative Score
- Neutral Score
- Positive Score
- Compound Score [-1,1]

In [ ]:
df.insert(loc=8,
          column='Negative Score',
          value=df['Polarity Score'])

df.insert(loc=9,
          column='Neutral Score',
          value=df['Polarity Score'])

df.insert(loc=10,
          column='Positive Score',
          value=df['Polarity Score'])

df.insert(loc=11,
          column='Compound Score',
          value=df['Polarity Score'])

In [ ]:
def negative_score(text):
    text = text['neg']
    return text

df['Negative Score'] = df['Negative Score'].apply(negative_score)

In [ ]:
def neutral_score(text):
    text = text['neu']
    return text

df['Neutral Score'] = df['Neutral Score'].apply(neutral_score)

In [ ]:
def positive_score(text):
    text = text['pos']
    return text

df['Positive Score'] = df['Positive Score'].apply(positive_score)

In [ ]:
def compound_score(text):
    text = text['compound']
    return text

df['Compound Score'] = df['Compound Score'].apply(compound_score)

In [ ]:
df.head()

Sentiment Analysis using TextBlob

In [69]:
df.insert(loc=12,
          column='Polarity Score_textblob',
          value=df['Adjusted Tweet'])

In [70]:
df.insert(loc=13,
          column='Subjectivity Score_textblob',
          value=df['Adjusted Tweet'])

In [71]:
from textblob import TextBlob

#Create a function to get the subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

#Create a function to get the polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

df['Polarity Score_textblob'] = df['Polarity Score_textblob'].apply(getPolarity)
df['Subjectivity Score_textblob'] = df['Subjectivity Score_textblob'].apply(getSubjectivity)


In [72]:
df.head()

id                       date             user  \
0  1477420789863436289  2022-01-01 23:25:40+00:00         anno1540   
1  1477414596424220679  2022-01-01 23:01:03+00:00   weather_odessa   
2  1477414332376010752  2022-01-01 23:00:00+00:00    AlArabiya_Eng   
3  1477409748572151809  2022-01-01 22:41:47+00:00    beatravelling   
4  1477409332820119552  2022-01-01 22:40:08+00:00  TornCurtain1991   

   user_followers               user_created  \
0            8838  2014-06-12 17:05:22+00:00   
1             119  2019-07-10 08:34:22+00:00   
2          927174  2009-02-28 08:31:32+00:00   
3            6329  2014-02-28 21:25:33+00:00   
4             677  2012-02-08 15:30:41+00:00   

                                    rendered_content  \
0  Lithuania will never abandon Ukraine, voluntee...   
1  #odessa #odesa #ukraine #одесса\nNow: 4.2°C\nT...   
2  After tough talk between Presidents Joe Biden ...   
3  The beach can be nice in the fall too 😊🇺🇦\n\n#...   
4  A note: Stepan #Bandera, DOB 01011909, was lea...   

                                      Adjusted Tweet  \
0  Lithuania will never abandon Ukraine, voluntee...   
1     #одесса\nNow: 4.2°C\nToday's Min: 4.2°C at ...   
2  After tough talk between Presidents Joe Biden ...   
3  The beach can be nice in the fall too 😊🇺🇦\n\n ...   
4  A note: Stepan , DOB 01011909, was leader of O...   

                                      Polarity Score  likes  retweets  ...  \
0  Lithuania will never abandon Ukraine, voluntee...      5         1  ...   
1     #одесса\nNow: 4.2°C\nToday's Min: 4.2°C at ...      0         0  ...   
2  After tough talk between Presidents Joe Biden ...      4         0  ...   
3  The beach can be nice in the fall too 😊🇺🇦\n\n ...      0         0  ...   
4  A note: Stepan , DOB 01011909, was leader of O...      1         2  ...   

   Polarity Score_textblob  Subjectivity Score_textblob  \
0                 0.000000                     0.000000   
1                 0.000000                     0.000000   
2                -0.194444                     0.666667   
3                 0.600000                     1.000000   
4                -0.100000                     0.033333   

                                            hashtags  lang media  \
0                           ['Lithuania', 'Ukraine']    en   NaN   
1           ['odessa', 'odesa', 'ukraine', 'одесса']    en   NaN   
2                              ['Russia', 'Ukraine']    en   NaN   
3  ['lanzheron', 'langeron', 'beach', 'odessa', '...    en   NaN   
4                             ['Bandera', 'Ukraine']    en   NaN   

  mentionedUsers tweet_length num_hashtags  num_mentions  num_urls  
0            NaN          132            2             0         0  
1            NaN          188            4             0         0  
2            NaN          277            2             0         0  
3            NaN          122            5             0         0  
4            NaN          278            2             0         0  

[5 rows x 22 columns]

## Further manipulating the tweet

Steps taken:
- Lowercase
- Punctuation
- Tokenization
- Stopword filtering
- Stemming

Changing all text to lowercase

In [73]:
def lowercase(text):    
    text = text.lower()
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(lowercase)

Removing all Punctuation

In [74]:
import string

def punctuation_remove(text):
    text = "".join([char for char in text if char not in string.punctuation])
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(punctuation_remove)

Tokenizing

In [75]:
from nltk import word_tokenize

def tokenize(text):
    text = word_tokenize(text)
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(tokenize)

Stopword Filtering

In [76]:
from nltk.corpus import stopwords

def remove_stopwords(text):
    stop_words = stopwords.words('english')
    text = [word for word in text if word not in stop_words]
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(remove_stopwords)

Stemming

In [77]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

def stem(text):
    stemmed = [porter.stem(word) for word in text]
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(stem)

## Months since creation of account relative to tweet

In [78]:
#Finding date of account creation in months
df.insert(loc=5,
          column='Date of Creation in months',
          value=df['user_created'])

In [79]:
from datetime import *

#returning the months of account creation
def account_creation(text):
    text = datetime.strptime(text, "%Y-%m-%d %H:%M:%S+00:00")
    year = str(text)[0:4]
    month = str(text)[5:7]
    total_months = (int(year)*12)+(int(month))
    return (total_months)

df['Date of Creation in months'] = df['Date of Creation in months'].apply(account_creation)

In [80]:
#Finding date of tweet in months
df.insert(loc=6,
          column='Date of Tweet in Months',
          value=df['date'])

In [81]:
from datetime import *

#return the year of tweet
df['Date of Tweet in Months'] = df['Date of Tweet in Months'].apply(account_creation)

In [82]:
#Calculating Months since creation of account relative to tweet
df.insert(loc=7,
          column='Months Since Creation of Account',
          value= (df['Date of Tweet in Months']-df['Date of Creation in months']))

## Time of Day

In [83]:
#create a column for hours:
df.insert(loc=2,
          column='hour of tweet',
          value=df['date'])

In [84]:
from datetime import *

#return the hour of the tweet
def hour(text):
    text = datetime.strptime(text, "%Y-%m-%d %H:%M:%S+00:00")
    hour = str(text.time())[0:2]
    return int(hour)

df['hour of tweet'] = df['hour of tweet'].apply(hour)

In [85]:
#insert column for the time of day
df.insert(loc=3,
          column='time of day',
          value=df['hour of tweet'])

In [86]:
#calculating the time of day
def time_of_day(text):  
    if ((text > 4) and (text < 8 )):
        return 'Early Morning'
    elif ((text > 8) and (text < 12 )):
        return 'Morning'
    elif ((text > 12) and (text < 16 )):
        return 'Noon'
    elif ((text > 16) and (text < 20 )):
        return 'Eve'
    elif ((text > 20) and (text < 24 )):
        return 'Night'
    elif ((text > 0) and (text < 4 )):
        return 'Late Night'
    
df['time of day'] = df['time of day'].apply(time_of_day)


In [87]:
#Creating counts using one hot encoding

#Early Morning Count
df.insert(loc=4,
          column='Early Morning Count',
          value=df['time of day'])

def early_morning_count(text):
    if text == 'Early Morning':
        return 1
    else:
        return 0

df['Early Morning Count'] = df['Early Morning Count'].apply(early_morning_count)

#Morning Count
df.insert(loc=5,
          column='Morning Count',
          value=df['time of day'])

def morning_count(text):
    if text == 'Morning':
        return 1
    else:
        return 0

df['Morning Count'] = df['Morning Count'].apply(morning_count)

#Noon count
df.insert(loc=6,
          column='Noon Count',
          value=df['time of day'])

def noon_count(text):
    if text == 'Noon':
        return 1
    else:
        return 0

df['Noon Count'] = df['Noon Count'].apply(noon_count)

#Eve count
df.insert(loc=7,
          column='Eve Count',
          value=df['time of day'])

def eve_count(text):
    if text == 'Eve':
        return 1
    else:
        return 0

df['Eve Count'] = df['Eve Count'].apply(eve_count)

#Night count
df.insert(loc=8,
          column='Night Count',
          value=df['time of day'])

def night_count(text):
    if text == 'Night':
        return 1
    else:
        return 0

df['Night Count'] = df['Night Count'].apply(night_count)

#Late Night count
df.insert(loc=9,
          column='Late Night Count',
          value=df['time of day'])

def late_night_count(text):
    if text == 'Late Night':
        return 1
    else:
        return 0

df['Late Night Count'] = df['Late Night Count'].apply(late_night_count)

## Video, GIF and Photo Count

Photo Count

In [88]:
#Creating a photo count column...
df.insert(loc=23,
          column='Photo Count',
          value=df['media'])

In [89]:
#Counting number of Photos in media column
#No need to tokenize
def photo_count(text):
    text = str(text)
    text = text.count('Photo')
    return text

df['Photo Count'] = df['Photo Count'].apply(photo_count)


Video Count

In [90]:
#Creating a video count column...
df.insert(loc=24,
          column='Video Count',
          value=df['media'])

In [91]:
#We need to tokenize the media column so that we can count how many videos there are...
from nltk import word_tokenize

def tokenize(text):
    text = str(text)
    text = word_tokenize(text)
    return text

df['Video Count'] = df['Video Count'].apply(tokenize)

In [92]:
#Counting number of Videos in media column
def video_count(text):
    text = text.count('Video')
    return text

df['Video Count'] = df['Video Count'].apply(video_count)

Gif Count

In [93]:
#Creating a GIF count column...
df.insert(loc=25,
          column='GIF Count',
          value=df['media'])

In [94]:
#We need to tokenize the media column so that we can count how many GIFs there are...
from nltk import word_tokenize

def tokenize(text):
    text = str(text)
    text = word_tokenize(text)
    return text

df['GIF Count'] = df['GIF Count'].apply(tokenize)

In [95]:
#Counting number of GIFs in media column
def gif_count(text):
    text = text.count('Gif')
    return text

df['GIF Count'] = df['GIF Count'].apply(gif_count)

# Topic Modelling

In [99]:
from sklearn.feature_extraction.text import CountVectorizer

# the vectorizer object will be used to transform text to vector form
vectorizer = CountVectorizer(max_df=0.9, min_df=25, token_pattern='\w+|\$[\d\.]+|\S+')

# apply transformation
tf = vectorizer.fit_transform(df['rendered_content']).toarray()

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names()

C:\Users\ekmho\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [105]:
from sklearn.decomposition import LatentDirichletAllocation

number_of_topics = 10

model = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)



In [ ]:
model.fit(tf)

In [101]:
def display_topics(model, feature_names, no_top_words):
    topic_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        topic_dict["Topic %d weights" % (topic_idx)]= ['{:.1f}'.format(topic[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
    return pd.DataFrame(topic_dict)

In [102]:
no_top_words = 10
display_topics(model, tf_feature_names, no_top_words)

Topic 0 words Topic 0 weights Topic 1 words Topic 1 weights Topic 2 words  \
0       #russia          4444.2            to          8580.2             .   
1      #ukraine          4225.6             .          5144.0           the   
2   #ukrainewar          2770.3           the          5122.3             ,   
3         #nato          2599.3           are          3123.4            of   
4       ukraine          2452.3      #ukraine          2958.3             a   
5            of          2009.2           and          2815.1            in   
6            to          1958.3            is          2739.3           and   
7             :          1771.1            in          2303.4            is   
8          #war          1690.4             ?          2272.3            to   
9          #usa          1486.4          they          2244.0      #ukraine   

  Topic 2 weights Topic 3 words Topic 3 weights        Topic 4 words  \
0         16105.8            to          3206.2          #ukrainewar   
1         14168.4      #ukraine          3129.1             #ukraine   
2          8724.2             :          2385.7                    |   
3          7758.3       ukraine          2124.5    #ukrainerussiawar   
4          7331.6        russia          1558.6              ukraine   
5          6768.1           for          1483.3                   of   
6          6095.3   #ukrainewar          1455.9    #standwithukraine   
7          6020.7          more          1293.3                    :   
8          5890.8             ,          1284.8  #ukraineunderattaсk   
9          4007.7         https          1221.8                  war   

  Topic 4 weights Topic 5 words Topic 5 weights Topic 6 words Topic 6 weights  \
0          2281.7           the         12974.1           the          9771.3   
1          2067.9            to          9585.8            of          7948.0   
2          1920.8             .          6419.5             ,          5967.3   
3          1763.0             ,          6020.6            in          5053.2   
4          1658.7            of          5728.2      #ukraine          3379.3   
5          1487.3      #ukraine          5348.1             .          2733.3   
6          1158.8             a          5152.9            on          2254.7   
7          1080.4           and          4947.9         https          2034.1   
8           997.5            in          4931.7   #ukrainewar          1941.0   
9           987.7            is          4138.8           and          1884.7   

          Topic 7 words Topic 7 weights Topic 8 words Topic 8 weights  \
0           #ukrainewar          6458.7           the         10135.7   
1              #ukraine          5063.9            of          6640.6   
2                 https          4430.1         https          6197.8   
3     #ukrainerussiawar          4284.3             .          5418.7   
4    #ukrainerussianwar          1949.5      #ukraine          5353.8   
5   #ukraineunderattack          1926.6   #ukrainewar          5108.3   
6               #russia          1825.5            in          4921.9   
7     #standwithukraine          1651.6       russian          3671.3   
8  #russianukrainianwar          1418.1       #russia          3275.8   
9                    in          1404.0        forces          2982.1   

  Topic 9 words Topic 9 weights  
0      #ukraine          3899.3  
1             !          3863.9  
2             ,          2889.1  
3           you          2751.3  
4   #ukrainewar          2089.6  
5             i          2065.5  
6         https          1985.9  
7             .          1552.9  
8          this          1256.7  
9           for          1215.8

In [103]:
from sklearn.decomposition import NMF

model_2 = NMF(n_components=10, random_state=0, alpha=.1, l1_ratio=.5)

model_2.fit(tf)

C:\Users\ekmho\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1477: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  warnings.warn(
C:\Users\ekmho\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1692: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


NMF(alpha=0.1, l1_ratio=0.5, n_components=10, random_state=0)

In [104]:
display_topics(model_2, tf_feature_names, no_top_words)

Topic 0 words Topic 0 weights Topic 1 words Topic 1 weights  \
0           the            26.7             .            20.5   
1            on             1.4           are             0.9   
2         https             1.1          this             0.9   
3          from             0.8             i             0.9   
4            by             0.6   #ukrainewar             0.8   
5           has             0.6          they             0.8   
6          with             0.5      #ukraine             0.7   
7         world             0.5            it             0.7   
8          that             0.5            be             0.6   
9            at             0.5            we             0.6   

        Topic 2 words Topic 2 weights Topic 3 words Topic 3 weights  \
0            #ukraine            13.1             ,            20.3   
1         #ukrainewar            10.9           but             0.5   
2             #russia             7.4           you             0.4   
3               https             7.0            on             0.4   
4   #ukrainerussiawar             4.5           not             0.4   
5             ukraine             4.2            as             0.4   
6                   :             3.6           are             0.4   
7             russian             2.7            's             0.3   
8  #ukrainerussianwar             2.1         &amp;             0.3   
9                  on             1.8             :             0.3   

  Topic 4 words Topic 4 weights Topic 5 words Topic 5 weights Topic 6 words  \
0            to            20.4            of            21.5            in   
1       ukraine             1.8        forces             1.0           war   
2           for             1.3         armed             0.7         &amp;   
3            be             1.2       ukraine             0.7     #ukraine.   
4      #ukraine             1.0        people             0.5           are   
5            us             1.0      invasion             0.5        region   
6        russia             0.8       russian             0.4       ukraine   
7             :             0.8            as             0.3       russian   
8            it             0.8           one             0.3           was   
9         &amp;             0.8         &amp;             0.3      military   

  Topic 6 weights Topic 7 words Topic 7 weights Topic 8 words Topic 8 weights  \
0            21.4             a            20.0           and            19.3   
1             0.9         https             1.6           for             2.0   
2             0.6            on             1.3           are             1.5   
3             0.5           for             1.2      #ukraine             1.5   
4             0.5          with             1.1          with             1.0   
5             0.5          that             0.9            on             0.8   
6             0.5           was             0.9            we             0.7   
7             0.4            as             0.9         their             0.6   
8             0.4            by             0.8           our             0.6   
9             0.3       russian             0.8           all             0.5   

  Topic 9 words Topic 9 weights  
0            is            16.2  
1            it             4.1  
2          this             4.0  
3      #ukraine             3.6  
4          that             3.1  
5             ?             2.7  
6           not             2.2  
7           for             1.7  
8          what             1.7  
9           war             1.7

## Creating our final Dataframe

Steps taken:
- Drop: media, date, user, polarity score, user_created, Date of Creation in months, Date of Tweet in Months, rendered_content.	

In [96]:
df.drop(['media','date','user','Polarity Score','user_created','Date of Creation in months','Date of Tweet in Months','rendered_content'], axis=1)

id  hour of tweet time of day  Early Morning Count  \
0      1477420789863436289             23       Night                    0   
1      1477414596424220679             23       Night                    0   
2      1477414332376010752             23       Night                    0   
3      1477409748572151809             22       Night                    0   
4      1477409332820119552             22       Night                    0   
...                    ...            ...         ...                  ...   
60128  1609034657231675393              3  Late Night                    0   
60129  1609032640664838145              3  Late Night                    0   
60130  1609025141333438465              3  Late Night                    0   
60131  1609020670486405121              2  Late Night                    0   
60132  1609017379853410304              2  Late Night                    0   

       Morning Count  Noon Count  Eve Count  Night Count  Late Night Count  \
0                  0           0          0            1                 0   
1                  0           0          0            1                 0   
2                  0           0          0            1                 0   
3                  0           0          0            1                 0   
4                  0           0          0            1                 0   
...              ...         ...        ...          ...               ...   
60128              0           0          0            0                 1   
60129              0           0          0            0                 1   
60130              0           0          0            0                 1   
60131              0           0          0            0                 1   
60132              0           0          0            0                 1   

       user_followers  ...  GIF Count Polarity Score_textblob  \
0                8838  ...          0                0.000000   
1                 119  ...          0                0.000000   
2              927174  ...          0               -0.194444   
3                6329  ...          0                0.600000   
4                 677  ...          0               -0.100000   
...               ...  ...        ...                     ...   
60128            2559  ...          0                0.000000   
60129            2250  ...          0                0.000000   
60130           69667  ...          0                0.000000   
60131             299  ...          0                0.600000   
60132             456  ...          0               -0.400000   

       Subjectivity Score_textblob  \
0                         0.000000   
1                         0.000000   
2                         0.666667   
3                         1.000000   
4                         0.033333   
...                            ...   
60128                     0.000000   
60129                     0.000000   
60130                     0.000000   
60131                     1.000000   
60132                     0.700000   

                                                hashtags  lang  \
0                               ['Lithuania', 'Ukraine']    en   
1               ['odessa', 'odesa', 'ukraine', 'одесса']    en   
2                                  ['Russia', 'Ukraine']    en   
3      ['lanzheron', 'langeron', 'beach', 'odessa', '...    en   
4                                 ['Bandera', 'Ukraine']    en   
...                                                  ...   ...   
60128  ['UkraineRussiaWar️', 'Russia', 'Ukraine', 'wa...    en   
60129  ['Venezuela', 'RussiaUkraineWar', 'Ukraine️', ...    en   
60130  ['Russia', 'Ukraine', 'NATO', 'Lavrov', 'Ukrai...    en   
60131  ['Ukraine️', 'Ukrainian', 'UkraineRussiaWar️',...    en   
60132  ['Ukraine', 'UkraineRussianWar', 'UkraineTerro...    en   

       mentionedUsers  tweet_length  num_hashtags  num_mentions  num_urls  
0                 NaN           132        